In [1]:
from tavily import TavilyClient

tavily_client = TavilyClient(api_key="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")
response = tavily_client.search("김치 맛집좀 추천")

print(response)

{'query': '김치 맛집좀 추천', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'http://rabbit.koreatimes.com/in/detail/676', 'title': '김치 맛있게 하는 곳 있나요? | 지식톡 - 케이타운 일번가', 'content': '우연히 김치를 보다가 댓글달아 봅니다 브래드김치가 많이 있는데 저는 노부랜드 한순이네 김치가 너무 맛있더라구요 조미료가 들어가지 않아서 숙성이될수록 깔끔', 'score': 0.60861784, 'raw_content': None}, {'url': 'https://www.reddit.com/r/korea/comments/1d6tbqc/i_need_kimchi_recommendations_from_amazon_never/?tl=ko', 'title': '아마존에서 김치 추천 좀 해줘. 미국에서 김치는 한 번도 안 사봐서 ...', 'content': 'Jun 3, 2024·코스트코/홀푸드/H마트에서 파는 건 대부분 깔끔하고 신선하잖아요. 그것도 맛있어요! 그리고 한국 슈퍼마켓에서 파는 거랑 비슷하죠. 저는 클리블랜드나', 'score': 0.59061456, 'raw_content': None}, {'url': 'https://www.milemoa.com/bbs/board/5382824', 'title': 'LA에서 구입할 수 있는 김치 추천해주실 수 있으실까요? - 마일모아', 'content': 'Oct 25, 2018·일반 김치는 종가집 또는 서울식 김치는 코스모스 제품. 찌개용으로는 농협김치. 좀 맵고 싼가격은 아니지만 또순이 김치 추천합니다. https://www.yelp', 'score': 0.5504388, 'raw_content': None}, {'url': 'https://www.reddit.com/r/FoodLosAngeles/comments/14fw71v/what_restauran

In [ ]:
from tavily import TavilyClient

tavily_client = TavilyClient(api_key="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")
response = tavily_client.extract(
    "https://en.wikipedia.org/wiki/Artificial_intelligence", 
    include_images =True,
    extract_depth="basic"
)
print(response)

{'results': [{'url': 'https://en.wikipedia.org/wiki/Artificial_intelligence', 'raw_content': 'Published Time: Tue, 02 Sep 2025 13:27:52 GMT\n\nArtificial intelligence - Wikipedia\n\n===============\n[Jump to content](https://en.wikipedia.org/wiki/Artificial_intelligence#bodyContent)\n\n- [x] Main menu \n\nMain menu\n\nmove to sidebar hide\n\n Navigation \n\n*   [Main page](https://en.wikipedia.org/wiki/Main_Page "Visit the main page [z]")\n*   [Contents](https://en.wikipedia.org/wiki/Wikipedia:Contents "Guides to browsing Wikipedia")\n*   [Current events](https://en.wikipedia.org/wiki/Portal:Current_events "Articles related to current events")\n*   [Random article](https://en.wikipedia.org/wiki/Special:Random "Visit a randomly selected article [x]")\n*   [About Wikipedia](https://en.wikipedia.org/wiki/Wikipedia:About "Learn about Wikipedia and how it works")\n*   [Contact us](https://en.wikipedia.org/wiki/Wikipedia:Contact_us "How to contact Wikipedia")\n\n Contribute \n\n*   [Help](ht

In [13]:
for r in response["results"]:
    print(r["url"])
    print(r.get("images", []))  # ← 이미지 URL 리스트


https://en.wikipedia.org/wiki/Artificial_intelligence
[]


In [15]:
from tavily import TavilyClient
from pprint import pprint

client = TavilyClient(api_key="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")

response = client.extract(
    urls=["https://en.wikipedia.org/wiki/Artificial_intelligence"],  # ← 가독성 위해 명시적/리스트 권장
    include_images=True,
    extract_depth="basic",   # 필요시 "advanced"
)

# 구조 확인(처음엔 꼭 찍어보세요)
pprint(response)
pprint(response["results"][0].keys())   # 어떤 키들이 있는지 확인

# ✅ 각 결과에서 URL과 이미지 리스트 꺼내기
for item in response.get("results", []):
    print("PAGE:", item.get("url"))
    print("IMAGES:", item.get("images", []))   # 없으면 [] 반환


{'failed_results': [],
 'request_id': '7bcee317-981f-41c0-9ecf-ef1c7316834c',
 'response_time': 0.02,
 'results': [{'images': ['https://en.wikipedia.org/static/images/icons/wikipedia.png',
                         'https://en.wikipedia.org/static/images/mobile/copyright/wikipedia-wordmark-en.svg',
                         'https://en.wikipedia.org/static/images/mobile/copyright/wikipedia-tagline-en.svg',
                         'https://upload.wikimedia.org/wikipedia/en/thumb/1/1b/Semi-protection-shackle.svg/20px-Semi-protection-shackle.svg.png',
                         'https://upload.wikimedia.org/wikipedia/commons/thumb/6/64/Dall-e_3_%28jan_%2724%29_artificial_intelligence_icon.png/120px-Dall-e_3_%28jan_%2724%29_artificial_intelligence_icon.png',
                         'https://upload.wikimedia.org/wikipedia/commons/thumb/e/e8/General_Formal_Ontology.svg/330px-General_Formal_Ontology.svg.png',
                         'https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Su

In [18]:
from sentence_transformers import CrossEncoder
import torch, re
from typing import List, Dict

# ★ 다국어 리랭커 권장 모델
CROSS_ENCODER_MODEL = "jinaai/jina-reranker-v2-base-multilingual"

# 1) 불필요 도메인(노이즈) 제거 규칙
BLOCKED_DOMAINS = [
    "instagram.com", "reddit.com", "forbes.com"  # SNS/커뮤니티/라면 등 오프라인 샵과 거리가 멀 가능성
]

# 2) 오프라인 샵 신호(가산점) 키워드
SHOP_TERMS = [
    "기념품", "굿즈", "기프트샵", "선물가게", "매장", "오프라인", "판매처",
    "souvenir", "gift shop", "store", "merch", "official shop"
]

# 3) 주소/장소 힌트(가산점)
PLACE_TERMS = [
    "서울", "Seoul", "종로", "용산", "명동", "DDP", "세종문화회관", "박물관", "뮤지엄", "Jongno", "Yongsan", "Myeong-dong"
]

# 4) 공신력 높은(또는 가능성 높은) 도메인 화이트리스트(가산점)
TRUSTED_DOMAINS = [
    "sejong", "sejongpac.or.kr", "visitseoul.net", "museum.go.kr", "muds", "seoul", "go.kr"
]

def domain_of(url: str) -> str:
    return url.split("/")[2] if "://" in url else url

def contains_any(text: str, terms: List[str]) -> int:
    t = text.lower()
    return sum(1 for k in terms if k.lower() in t)

def looks_like_address(text: str) -> bool:
    # 아주 단순한 주소/운영시간 패턴(필요시 강화)
    return bool(re.search(r"(서울|Seoul).*(구|로|길|동|센터|Center|Museum)", text)) or \
           bool(re.search(r"(\d{1,2}:\d{2}\s*[-~]\s*\d{1,2}:\d{2}|open|hours|운영시간|영업시간)", text, re.I))

def preprocess_results(results: List[Dict]) -> List[Dict]:
    """인스타/레딧/포브스 같은 노이즈를 우선 제거"""
    cleaned = []
    for r in results:
        url = r.get("url","")
        dom = domain_of(url)
        if any(b in dom for b in BLOCKED_DOMAINS):
            continue
        cleaned.append(r)
    return cleaned

def to_text_blob(r: Dict) -> str:
    # title + content + raw_content + url을 하나의 문자열로
    return " | ".join([
        r.get("title",""),
        r.get("content",""),
        (r.get("raw_content") or ""),
        r.get("url","")
    ])

def offline_boost(item: Dict) -> float:
    """오프라인 샵에 유리한 신호에 가산점"""
    text = (item.get("title","") + " " + item.get("content","") + " " + (item.get("raw_content") or ""))
    url  = item.get("url","")
    dom  = domain_of(url)

    bonus = 0.0
    bonus += 0.6 * contains_any(text, SHOP_TERMS)
    bonus += 0.4 * contains_any(text, PLACE_TERMS)
    if looks_like_address(text):
        bonus += 0.6
    if any(td in dom for td in TRUSTED_DOMAINS):
        bonus += 0.5
    # 너무 상업적 온라인몰/뉴스만 같으면 약간 감점도 가능 (원하면 추가)
    return bonus

def rerank_offline_shops(query: str, results: List[Dict], top_n: int = 5, device: str = None) -> List[Dict]:
    # 0) 노이즈 도메인 제거
    pool = preprocess_results(results)
    if not pool:
        return []

    # 1) CrossEncoder 로드
    model = CrossEncoder(CROSS_ENCODER_MODEL, device=device or ("cuda" if torch.cuda.is_available() else "cpu"))

    # 2) 쿼리-문서 쌍 만들기
    pairs = [[query, to_text_blob(r)] for r in pool]

    # 3) 의미유사도 점수
    ce_scores = model.predict(pairs)  # float 배열
    for r, s in zip(pool, ce_scores):
        r["_ce_score"] = float(s)

    # 4) 최종 점수 = 의미유사도 + 오프라인샵 가산점 + Tavily 기본 score(있으면)
    for r in pool:
        base = float(r.get("score", 0.0))
        boost = offline_boost(r)
        r["_final_score"] = 0.6*r["_ce_score"] + 0.3*boost + 0.1*base

    # 5) 정렬 후 상위 N
    pool.sort(key=lambda x: x["_final_score"], reverse=True)
    return pool[:min(top_n, len(pool))]


c:\tavily_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
